In [ ]:
# Notebook: 3-Live-Prediction.ipynb (Final Version with Video Display Fix)

import os
import cv2
import numpy as np
import tensorflow as tf
from IPython.display import HTML, display
from base64 import b64encode # Import the necessary library for embedding

# --- CONFIGURATION ---
BASE_DIR = "D:/Distracted_Driving_Detection_Project"
RESULTS_DIR = os.path.join(BASE_DIR, "Model_Results")
MODEL_PATH = os.path.join(RESULTS_DIR, 'best_model.keras')

CLASSES_LIST = ["Normal_Driving", "Looking_Left", "Looking_Right", "Using_Phone"]
IMAGE_HEIGHT, IMAGE_WIDTH = 64, 64
SEQUENCE_LENGTH = 20

# --- PREDICTION FUNCTIONS ---
def frames_extraction(video_path):
    frames_list = []
    video_reader = cv2.VideoCapture(video_path)
    video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))
    skip_frames_window = max(int(video_frames_count / SEQUENCE_LENGTH), 1)
    for frame_counter in range(SEQUENCE_LENGTH):
        video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)
        success, frame = video_reader.read()
        if not success: break
        resized_frame = cv2.resize(frame, (IMAGE_WIDTH, IMAGE_HEIGHT))
        frames_list.append(resized_frame / 255.0)
    video_reader.release()
    return frames_list

def predict_on_video(video_path):
    if not os.path.exists(MODEL_PATH):
        return f"Error: Model not found at {MODEL_PATH}"
    
    model = tf.keras.models.load_model(MODEL_PATH)
    video_frames = frames_extraction(video_path)
    
    if len(video_frames) != SEQUENCE_LENGTH:
        return f"Error: The video is too short or could not be processed."

    frames_for_prediction = np.expand_dims(video_frames, axis=0)
    predictions = model.predict(frames_for_prediction)
    predicted_class_name = CLASSES_LIST[np.argmax(predictions)]
    confidence = np.max(predictions) * 100
    
    print(f"✅ PREDICTION: '{predicted_class_name}' ({confidence:.2f}%)")
    
    # --- THIS BLOCK IS NOW CORRECTED TO EMBED THE VIDEO ---
    # Read the video file in binary mode
    mp4 = open(video_path,'rb').read()
    # Encode it in base64 and create a data URL
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    
    # Return an HTML object with the embedded video data
    return HTML(f"""
    <video width=400 controls>
          <source src="{data_url}" type="video/mp4">
    </video>
    """)
    # --- END OF FIX ---

# ==============================================================================
# Your test video path
# ==============================================================================
test_video_path = "D:/Distracted_Driving_Detection_Project/test_video.mp4"

if os.path.exists(test_video_path):
    display(predict_on_video(test_video_path))
else:
    print(f"Test video not found at: {test_video_path}")